In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

# 設定影像的預處理步驟
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # 調整影像大小
    transforms.ToTensor(),  # 轉換為Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # 正規化
])

# 使用 ImageFolder 加載資料
train_dataset = ImageFolder(root='./dataset', transform=transform)

# 使用 DataLoader 來生成訓練數據批次
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


KeyboardInterrupt: 

In [2]:
# 使用預訓練的 ResNet 模型
model = models.resnet18(pretrained=True)

# 修改最後一層，適應 12 個類別
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 移動模型到 GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\tools\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\tools\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\ECA-2/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:24<00:00, 1.88MB/s]


In [4]:
# 定義損失函數
criterion = nn.CrossEntropyLoss()

# 定義優化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [5]:
# 訓練模型
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # 設置模型為訓練模式
    running_loss = 0.0

    for images, labels in train_loader:
        # 將影像和標籤移動到 GPU
        images = images.to(device)
        labels = labels.to(device)

        # 清零梯度
        optimizer.zero_grad()

        # 前向傳播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向傳播和優化
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

print("訓練完成")


Epoch [1/10], Loss: 0.177172537586783
Epoch [2/10], Loss: 0.02443093727065419
Epoch [3/10], Loss: 0.06286261816299676
Epoch [4/10], Loss: 0.025458205358676874
Epoch [5/10], Loss: 0.04867334687951858
Epoch [6/10], Loss: 0.022479497192383763
Epoch [7/10], Loss: 0.02627527377323635
Epoch [8/10], Loss: 0.023305143003571536
Epoch [9/10], Loss: 0.01678767153557082
Epoch [10/10], Loss: 0.030417677402303354
訓練完成


In [6]:
# 保存模型權重
torch.save(model.state_dict(), 'model.pth')


In [20]:
# 加載模型權重
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('model.pth'))
model = model.to(device)
model.eval()  # 設置模型為評估模式

# 測試圖片的預處理
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# 測試單張圖片
from PIL import Image
img = Image.open('./OIP.jpg')
img = test_transform(img).unsqueeze(0)  # 增加 batch 維度
img = img.to(device)

# 推理
with torch.no_grad():
    outputs = model(img)
    _, predicted = outputs.max(1)
    print(f"預測結果: {predicted.item()}")


預測結果: 2


C:\Users\ECA-2\AppData\Local\Temp\ipykernel_13224\1243484363.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))
